In [1]:
import os
import time
import sys
import random
import psutil

import json
import copy
import numpy as np
import pandas as pd
from collections import Counter

from skimage.measure import label
from scipy.ndimage.morphology import distance_transform_edt

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.nn import init
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
from collections import OrderedDict
from sklearn.metrics import confusion_matrix

from utils import create_conf_matrix2, split_list, convert_to_binary_mask, classScores
#if there is a utils folder change name above
from models import UNet2
from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box
from loss_functions import loss_function_iou

random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file
#image_folder = '../../DLSP20Dataset/data'
#annotation_csv = '../../DLSP20Dataset/data/annotation.csv'

#azure
image_folder = '../../data'
annotation_csv = '../../data/annotation.csv'


#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'

torch.cuda.is_available()

True

In [2]:
# You shouldn't change the unlabeled_scene_index
# The first 106 scenes are unlabeled
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)

train_scene_index = np.random.choice(labeled_scene_index, int(np.ceil(0.9*len(labeled_scene_index))))

test_scene_index = labeled_scene_index[np.isin(labeled_scene_index, train_scene_index, invert=True)]

val_scene_index, test_scene_index = split_list(test_scene_index)


transform=torchvision.transforms.Compose([torchvision.transforms.Resize((100,100)),
                                          torchvision.transforms.ToTensor(),
                              #torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])

batch_size = 1

labeled_valset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=val_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )

valloader = torch.utils.data.DataLoader(labeled_valset, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=0, 
                                          collate_fn=collate_fn)

labeled_testset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=test_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )

testloader = torch.utils.data.DataLoader(labeled_testset, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=0, 
                                          collate_fn=collate_fn)

In [3]:
def validate_model_BB(model, num_points, loss_function, device=None, img_w=100, img_h=100):
    threshold_values = np.linspace(0.01,0.99,num_points)
    correct = [0]*num_points
    total = [0]*num_points
    accuracy_list = [None]*num_points
    threat_score_list = [None]*num_points
    conf_matrices = [torch.zeros(2,2).to(device),torch.zeros(2,2).to(device)]*num_points
    max_threat_score = 0
    opt_threshold = 0.5
    for k,threshold in enumerate(threshold_values):  
        with torch.no_grad():
            model.eval()
            test_loss = 0
            for batch_idx, data in enumerate(valloader):
                sample, target, road_image, extra = data
                batch_size = len(road_image)
                x = torch.zeros((batch_size,1,800,800))
                #x[:,0,:,:] = 1.0*torch.stack(road_image).reshape(-1, 800, 800)
                x[:,0,:,:] = torch.zeros((batch_size,800,800))
                for i in range(batch_size):
                    for cat, bb in zip(target[i]['category'], target[i]['bounding_box']):
                        x[i,0,:,:] = 1.0*convert_to_binary_mask(bb)
                x = x.to(device) 
                y = torch.stack(sample).reshape(6,-1,3,img_w,img_h).to(device)

                # ===================forward=====================
                x_hat = model(y)
                test_loss += loss_function(x_hat, x).item()
                # =====================log=======================

                i = 0
                correct[k] += (x_hat[:,i,:,:]>threshold).eq(
                    (x[:,i,:,:]==1).data.view_as((
                        x_hat[:,i,:,:]>threshold))).cpu().sum().item()
                total[k] += x[:,i,:,:].nelement()

                conf_matrices[k] += create_conf_matrix2(1*(x[:,i,:,:]==1), 1*(x_hat[:,i,:,:]>threshold), device=device)
                #print('='*100)


        accuracy_list[k] = 100. * correct[k] / total[k]
        TP, TN, FP, FN = classScores(conf_matrices[k])
        threat_score_list[k] = TP[1]*1.0/(TP[1]+FP[1]+FN[1])
        if threat_score_list[k] > max_threat_score:
            print('Updating threat score from {} to {}'.format(threat_score_list[k],max_threat_score))
            max_threat_score = threat_score_list[k]
            opt_threshold = threshold

        print("""\nValidation set: 
        Average loss: {:.9f}, 
        Accuracy BB: {}/{} ({:.9f}%) ,
        BB: 
        TP {} 
        TN {}
        FP {}
        FN {}
        Threat Score {}""".format(
            test_loss, 
            correct[k], total[k], accuracy_list[k],
             TP[1], TN[1], FP[1], FN[1], threat_score_list[k]))

        # ===================log========================
        test_loss /= len(testloader.dataset)
        print(f'====> Validation set loss: {test_loss:.9f}')
    return opt_threshold, max_threat_score

In [7]:
def validate_model_Road(model, num_points, loss_function, device=None, img_w=100, img_h=100, low=0.01, high=0.99):
    threshold_values = np.linspace(low,high,num_points)
    correct = [0]*num_points
    total = [0]*num_points
    accuracy_list = [None]*num_points
    threat_score_list = [None]*num_points
    conf_matrices = [torch.zeros(2,2).to(device),torch.zeros(2,2).to(device)]*num_points
    max_threat_score = 0
    opt_threshold = 0.5
    for k,threshold in enumerate(threshold_values):  
        with torch.no_grad():
            model.eval()
            test_loss = 0
            for batch_idx, data in enumerate(valloader):
                sample, target, road_image, extra = data
                batch_size = len(road_image)
                x = torch.zeros((batch_size,1,800,800))
                x[:,0,:,:] = 1.0*torch.stack(road_image).reshape(-1, 800, 800)
                #x[:,0,:,:] = torch.zeros((batch_size,800,800))
                #for i in range(batch_size):
                #    for cat, bb in zip(target[i]['category'], target[i]['bounding_box']):
                #        x[i,0,:,:] = 1.0*convert_to_binary_mask(bb)
                x = x.to(device) 
                y = torch.stack(sample).reshape(6,-1,3,img_w,img_h).to(device)

                # ===================forward=====================
                x_hat = model(y)
                test_loss += loss_function(x_hat, x, device=device).item()
                # =====================log=======================

                i = 0
                correct[k] += (x_hat[:,i,:,:]>threshold).eq(
                    (x[:,i,:,:]==1).data.view_as((
                        x_hat[:,i,:,:]>threshold))).cpu().sum().item()
                total[k] += x[:,i,:,:].nelement()

                conf_matrices[k] += create_conf_matrix2(1*(x[:,i,:,:]==1), 1*(x_hat[:,i,:,:]>threshold), device=device)
                #print('='*100)


        accuracy_list[k] = 100. * correct[k] / total[k]
        TP, TN, FP, FN = classScores(conf_matrices[k])
        threat_score_list[k] = TP[1]*1.0/(TP[1]+FP[1]+FN[1])
        if threat_score_list[k] > max_threat_score:
            print('Updating threat score from {} to {}'.format(threat_score_list[k],max_threat_score))
            max_threat_score = threat_score_list[k]
            opt_threshold = threshold

        print("""\nValidation set: 
        Average loss: {:.9f}, 
        Accuracy BB: {}/{} ({:.9f}%) ,
        BB: 
        TP {} 
        TN {}
        FP {}
        FN {}
        Threat Score {}""".format(
            test_loss, 
            correct[k], total[k], accuracy_list[k],
             TP[1], TN[1], FP[1], FN[1], threat_score_list[k]))

        # ===================log========================
        test_loss /= len(testloader.dataset)
        print(f'====> Validation set loss: {test_loss:.9f}')
    return opt_threshold, max_threat_score

In [5]:
model = UNet2(1)
model.load_state_dict(torch.load('models/unet_single_2_Roadmap_model_final.pth'))
model.eval()

UNet2(
  (dconv_down1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.1)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dconv_down2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.1)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dconv_down3): Sequential(
    (0): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Lea

In [6]:
device = 'cpu'

In [8]:
validate_model_Road(model, 2, loss_function_iou, device=device, img_w=100, img_h=100, low=0.49, high=0.51)

Confusion matrix
 tensor([[2.0730e+08, 4.1822e+07],
        [2.6475e+07, 1.2760e+08]])
Class 0
TP 207298048.0, TN 127604936.0, FP 26474900.0, FN 41822024.0
Class 1
TP 127604936.0, TN 207298048.0, FP 41822024.0, FN 26474900.0
Updating threat score from 0.6513717562737129 to 0

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 334903097/403200000 (83.061283978%) ,
        BB: 
        TP 127604936.0 
        TN 207298048.0
        FP 41822024.0
        FN 26474900.0
        Threat Score 0.6513717562737129
====> Validation set loss: -0.592152242


/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Confusion matrix
 tensor([[2.0734e+08, 4.1780e+07],
        [2.6485e+07, 1.2760e+08]])
Class 0
TP 207339808.0, TN 127595048.0, FP 26484814.0, FN 41780472.0
Class 1
TP 127595048.0, TN 207339808.0, FP 41780472.0, FN 26484814.0
Updating threat score from 0.6514593541798069 to 0.6513717562737129

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 334934697/403200000 (83.069121280%) ,
        BB: 
        TP 127595048.0 
        TN 207339808.0
        FP 41780472.0
        FN 26484814.0
        Threat Score 0.6514593541798069
====> Validation set loss: -0.592152242


/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


(0.51, 0.6514593541798069)

In [ ]:
validate_model_Road(model=model, 
                  num_points=49, 
                  loss_function=loss_function_iou, 
                  device=device)

Confusion matrix
 tensor([[2.0417e+08, 4.4949e+07],
        [2.5835e+07, 1.2824e+08]])
Class 0
TP 204171312.0, TN 128244784.0, FP 25835092.0, FN 44948768.0
Class 1
TP 128244784.0, TN 204171312.0, FP 44948768.0, FN 25835092.0
Updating threat score from 0.6443534156692223 to 0

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 332416140/403200000 (82.444479167%) ,
        BB: 
        TP 128244784.0 
        TN 204171312.0
        FP 44948768.0
        FN 25835092.0
        Threat Score 0.6443534156692223
====> Validation set loss: -0.592152242


/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Confusion matrix
 tensor([[2.0527e+08, 4.3854e+07],
        [2.6050e+07, 1.2803e+08]])
Class 0
TP 205266144.0, TN 128029696.0, FP 26050178.0, FN 43853972.0
Class 1
TP 128029696.0, TN 205266144.0, FP 43853972.0, FN 26050178.0
Updating threat score from 0.6468307592072179 to 0.6443534156692223

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 333295826/403200000 (82.662655258%) ,
        BB: 
        TP 128029696.0 
        TN 205266144.0
        FP 43853972.0
        FN 26050178.0
        Threat Score 0.6468307592072179
====> Validation set loss: -0.592152242


/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Confusion matrix
 tensor([[4.0983e+08, 8.8411e+07],
        [5.1963e+07, 2.5620e+08]])
Class 0
TP 409829248.0, TN 256196608.0, FP 51963180.0, FN 88410816.0
Class 1
TP 256196608.0, TN 409829248.0, FP 88410816.0, FN 51963180.0

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 333609863/403200000 (82.740541419%) ,
        BB: 
        TP 256196608.0 
        TN 409829248.0
        FP 88410816.0
        FN 51963180.0
        Threat Score 0.6460302734702023
====> Validation set loss: -0.592152242


...

/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Confusion matrix
 tensor([[2.6846e+09, 5.5392e+08],
        [3.4196e+08, 1.6611e+09]])
Class 0
TP 2684641280.0, TN 1661080576.0, FP 341955584.0, FN 553924096.0
Class 1
TP 1661080576.0, TN 2684641280.0, FP 553924096.0, FN 341955584.0

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 334919039/403200000 (83.065237847%) ,
        BB: 
        TP 1661080576.0 
        TN 2684641280.0
        FP 553924096.0
        FN 341955584.0
        Threat Score 0.6496309718159342
====> Validation set loss: -0.592152242


/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Confusion matrix
 tensor([[2.6867e+09, 5.5190e+08],
        [3.4237e+08, 1.6607e+09]])
Class 0
TP 2686665984.0, TN 1660669184.0, FP 342368512.0, FN 551897600.0
Class 1
TP 1660669184.0, TN 2686665984.0, FP 551897600.0, FN 342368512.0
Updating threat score from 0.6499848299876476 to 0.6498585329641037

Validation set: 
        Average loss: -373.055912412, 
        Accuracy BB: 334950851/403200000 (83.073127728%) ,
        BB: 
        TP 1660669184.0 
        TN 2686665984.0
        FP 551897600.0
        FN 342368512.0
        Threat Score 0.6499848299876476
====> Validation set loss: -0.592152242


/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1587428207430/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
